In [2]:
import os
import requests
import json
import openai
import openpyxl
import pandas as pd
import spacy
from spacy.lang.en import English

openai.api_key = "Your API key"
openai.api_base =  "https://YOUR_RESOURCE_NAME.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future
deployment_name='YOUR API DEPLOYMENT NAME' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'how are u \n'
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text) 

Sending a test completion job
how are u 
I'm doing great, thanks for asking!


In [3]:
# read csv
df = pd.read_csv(r'E:\Analysis With Azure Open AI - Chat GPT\input.csv')
print(df)

                                          Review Title  \
0         Another Midrange killer Smartphone by Xiaomi   
1                                vry small size mobile   
2         Full display not working in all application.   
3                                      Value for Money   
4                              Not worth for the money   
..                                                 ...   
275  Cemera quality,face unlock most important in t...   
276                                   Mi is best phone   
277                                     Its a OK Phone   
278                                              Redmi   
279                           Not worth for the price.   

                Customer name              Rating                  Date  \
0           Rishikumar Thakur  4.0 out of 5 stars     on 1 October 2018   
1                     Raza ji  3.0 out of 5 stars  on 15 September 2018   
2               Vaibhav Patel  3.0 out of 5 stars  on 18 September 2018   
3  

In [4]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of                                           Review Title  \
0         Another Midrange killer Smartphone by Xiaomi   
1                                vry small size mobile   
2         Full display not working in all application.   
3                                      Value for Money   
4                              Not worth for the money   
..                                                 ...   
275  Cemera quality,face unlock most important in t...   
276                                   Mi is best phone   
277                                     Its a OK Phone   
278                                              Redmi   
279                           Not worth for the price.   

                Customer name              Rating                  Date  \
0           Rishikumar Thakur  4.0 out of 5 stars     on 1 October 2018   
1                     Raza ji  3.0 out of 5 stars  on 15 September 2018   
2               Vaibhav Patel  3.0 

In [12]:
#Sentiment_for_each_comment
def CommentReview(comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"what type of comment - {comments} it is negative, positive or neutral?"),
        max_tokens=20
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

cols = ['Category', 'Comments', 'ReviewType']
df2 = pd.DataFrame(columns=cols)
Sentiment_for_each_comment = pd.DataFrame(columns=cols)

for ind in df.index:
    cat = df['Category'][ind]
    com = df['Comments'][ind]
    df_new_row = pd.DataFrame({'Category': df['Category'][ind], 'Comments':df['Comments'][ind], 'ReviewType': CommentReview(df['Comments'][ind])}, index=[0])
    Sentiment_for_each_comment = pd.concat([Sentiment_for_each_comment, df_new_row])
    

In [13]:
df_grouped = df.groupby("Category")["Comments"].apply(" ".join).reset_index()
df_grouped = df_grouped.rename(columns={"Comments": "Aggregated_Comments"})
df_grouped

,Category,Aggregated_Comments
0,Battery,"Thanks alot Amazon,ecom & seller.perfect produ..."
1,Camera,Front camera is poor rest things are good Fron...
2,Delivery,Nice product. Delivery guy also. Nice product....
3,Display,Another Midrange killer Smartphone by Xiaomi\n...
4,Others,All ok but vry small size mobile Quite good wo...


In [14]:
#category wise summariszation of comments

nlp = spacy.load("en_core_web_sm")

def text_to_chunks(text):
  chunks = [[]]
  chunk_total_words = 0

  sentences = nlp(text)

  for sentence in sentences.sents:
    chunk_total_words += len(sentence.text.split(" "))

    if chunk_total_words > 1000:
      chunks.append([])
      chunk_total_words = len(sentence.text.split(" "))

    chunks[len(chunks)-1].append(sentence.text)
  
  return chunks

def Comment_category(Comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"Give Summariaztion for the following comments:{Comments}"),
        max_tokens=200
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

print("Category wise Summarization of comments:")

cols = ['Category','Summarized Comment']
summariszation_of_comments = pd.DataFrame(columns=cols)

for ind in df_grouped.index:
    cat = df_grouped['Category'][ind]
    com = df_grouped['Aggregated_Comments'][ind]
    chunks = text_to_chunks(com)
    for chunk in chunks:
      chunk_summaries = []
      chunk_summary = Comment_category(" ".join(chunk))
      chunk_summaries.append(chunk_summary)
    summary = " ".join(chunk_summaries)
    print(df_grouped['Category'][ind], ":"+"\n"+Comment_category(summary), end="\n")
    df_new_row = pd.DataFrame({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)}, index=[0])
    summariszation_of_comments = pd.concat([summariszation_of_comments, df_new_row])

Category wise Summarization of comments:
Battery :
The Xiaomi 6 Pro is a decent mobile phone that offers a slim design, good sound quality, light weight, and good battery back-up, however there are some battery and charging problems, lagging, hanging, and other issues. Older models provide features such as length of USB cable and front flash that the Pro does not, leading to customers having slightly lower expectations when compared to the price for the 3GB/32GB version.
Camera :
This phone has unsatisfactory camera quality and an aged processor, along with some satisfactory features such as facial recognition and battery life. Unfortunately, its premium price tag is not justified with its low-quality cameras and poor design.
Delivery :
This customer was pleased with the quality of the product, the level of power, Amazon's prompt delivery, and value for money. However, they were disappointed that a charger was not included and that delivery was late.
Display :
This product has many adv

In [22]:
file_name = 'output.xlsx'
# Load the workbook
writer = pd.ExcelWriter('output.xlsx', engine='openpyxl') 
workbook = openpyxl.Workbook()
writer.book = workbook

# Write the DataFrame to a new sheet
Sentiment_for_each_comment.to_excel(writer, index=False, sheet_name='Sentiment for each comment')
summariszation_of_comments.to_excel(writer, index=False, sheet_name='summariszation of comments')

# Save the workbook
writer.save()
print('DataFrame is written to Excel File successfully.')

C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\2252963973.py:5: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = workbook
C:\Users\Yatin\AppData\Local\Temp\ipykernel_15220\2252963973.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


DataFrame is written to Excel File successfully.
